<a href="https://colab.research.google.com/github/M-T00/NLP_Codes/blob/main/RoBERTa_Examples(HuggingFaceCodesTest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers
from transformers import pipeline
#!pip install datasets

In [ ]:
# from datasets import load_dataset
# load_dataset("wikipedia", "20220301.en")

unmasker = pipeline('fill-mask', model='roberta-base')
unmasker("Hello I'm a <mask> model.")

[{'score': 0.33065226674079895,
  'token': 2943,
  'token_str': ' male',
  'sequence': "Hello I'm a male model."},
 {'score': 0.04655414819717407,
  'token': 2182,
  'token_str': ' female',
  'sequence': "Hello I'm a female model."},
 {'score': 0.042330026626586914,
  'token': 2038,
  'token_str': ' professional',
  'sequence': "Hello I'm a professional model."},
 {'score': 0.03721672669053078,
  'token': 2734,
  'token_str': ' fashion',
  'sequence': "Hello I'm a fashion model."},
 {'score': 0.03253656625747681,
  'token': 1083,
  'token_str': ' Russian',
  'sequence': "Hello I'm a Russian model."}]

In [ ]:
# "roberta-base" example
# Load model directly
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base")
classifier = pipeline("fill-mask", model="roberta-base")

text = "Now we gonna talking about <mask>."
encoded_input = tokenizer(text, padding = True, truncation = True, max_length = 512, return_tensors = "pt")

with torch.no_grad():
  outputs = model(**encoded_input)
  print(outputs)
  predictions = F.softmax(outputs.logits, dim = 1)
  print(predictions)
  labels = torch.argmax(predictions, dim = 1)
  print(labels)

MaskedLMOutput(loss=None, logits=tensor([[[34.1364, -3.9481, 18.2947,  ...,  2.6859,  5.6242, 10.9585],
         [ 0.2017, -4.5075, 14.8015,  ..., -3.7292, -1.8288,  3.8294],
         [ 0.1122, -4.4391,  8.2179,  ..., -4.6498, -4.6063,  1.9997],
         ...,
         [-4.4495, -3.7065,  2.7289,  ..., -7.0836, -7.2269, -1.4859],
         [16.5499, -4.9931, 18.2845,  ..., -1.6008,  1.0282,  5.6850],
         [10.8109, -4.4583, 28.4147,  ..., -3.0777, -3.1588,  5.5499]]]), hidden_states=None, attentions=None)
tensor([[[1.0000e+00, 1.5396e-01, 4.0265e-05,  ..., 7.6196e-01,
          9.8561e-01, 9.8936e-01],
         [1.8294e-15, 8.7994e-02, 1.2241e-06,  ..., 1.2471e-03,
          5.7133e-04, 7.9289e-04],
         [1.6729e-15, 9.4219e-02, 1.6929e-09,  ..., 4.9668e-04,
          3.5534e-05, 1.2722e-04],
         ...,
         [1.7471e-17, 1.9602e-01, 6.9952e-12,  ..., 4.3563e-05,
          2.5855e-06, 3.8978e-06],
         [2.3029e-08, 5.4144e-02, 3.9855e-05,  ..., 1.0477e-02,
          9.9

In [ ]:
# RobertaTokenizer
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokenizer("Hello world")["input_ids"]

tokenizer(" Hello world")["input_ids"]
# OR
# from transformers import RobertaTokenizerFast

# tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
# tokenizer("Hello world")["input_ids"]

# tokenizer(" Hello world")["input_ids"]

[0, 20920, 232, 2]

In [ ]:
#RobertaForCausalLM
from transformers import AutoTokenizer, RobertaForCausalLM, AutoConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
config = AutoConfig.from_pretrained("roberta-base")
config.is_decoder = True
model = RobertaForCausalLM.from_pretrained("roberta-base", config=config)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.logits
print(prediction_logits)

tensor([[[27.7073, -4.6183, 25.2725,  ...,  0.9433,  2.5662, 11.1328],
         [12.9955, -3.3435, 32.1472,  ...,  2.9392,  0.1569,  9.3820],
         [ 9.7384, -3.5373, 21.8191,  ...,  0.4943, -0.2668,  6.7838],
         ...,
         [ 5.6973, -4.3181, 21.6466,  ..., -3.2017, -4.4630,  4.5801],
         [ 4.9998, -4.6587, 15.6737,  ..., -3.9533, -4.5533,  2.9168],
         [24.6395, -4.3410, 23.1517,  ...,  0.3120,  2.7200,  9.1151]]],
       grad_fn=<ViewBackward0>)


In [ ]:
# RobertaForMaskedLM
from transformers import AutoTokenizer, RobertaForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

inputs = tokenizer("The capital of France is <mask>.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of <mask>
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-<mask> tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

0.1

In [ ]:
# Example of single-label classification:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion", num_labels=num_labels)

labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss
print(round(loss.item(), 2))

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

0.08


In [ ]:
# Example of multi-label classification:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion", problem_type="multi_label_classification")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion", num_labels=num_labels, problem_type="multi_label_classification"
)

labels = torch.sum(
    torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels), dim=1
).to(torch.float)
loss = model(**inputs, labels=labels).loss

In [ ]:
# RobertaForTokenClassification
from transformers import AutoTokenizer, RobertaForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
model = RobertaForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")

inputs = tokenizer(
    "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt"
)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
predicted_tokens_classes

labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)

tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/849 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

0.01

In [ ]:
# RobertaForQuestionAnswering
from transformers import AutoTokenizer, RobertaForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# target is "nice puppet"
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
round(loss.item(), 2)

0.86